[@LorenaABarba](https://twitter.com/LorenaABarba)

12 steps to Navier-Stokes
=====
## <font color='blue'> 나비어-스톡스 의 12 단계
***

This lesson complements the first interactive module of the online [CFD Python](https://github.com/barbagroup/CFDPython) class, by Prof. Lorena A. Barba, called **12 Steps to Navier-Stokes.** It was written with BU graduate student Gilbert Forsyth.

Array Operations with NumPy
----------------
### <font color='blue'>NumPy를 이용한 배열작업

For more computationally intensive programs, the use of built-in Numpy functions can provide an  increase in execution speed many-times over.  As a simple example, consider the following equation:
<font color='red'> 계산 집약적인 프로그램의 경우, 내장된 Numpy 함수를 사용하면 실행 속도가 향상됩니다. 간단한 예로서 아래의 방정식을 확인해보세요. </font>


$$u^{n+1}_i = u^n_i-u^n_{i-1}$$

Now, given a vector $u^n = [0, 1, 2, 3, 4, 5]\ \ $   we can calculate the values of $u^{n+1}$ by iterating over the values of $u^n$ with a for loop.  
<font color='red'>주어진 벡터 $u^n = [0, 1, 2, 3, 4, 5]\ \ $으로 $u^n$ 값을 **for** 루프로 반복하여 $u^{n+1}$ 값을 계산할수 있습니다. 

In [12]:
import numpy

In [13]:
u = numpy.array((0, 1, 2, 3, 4, 5))

for i in range(1, len(u)):
    print(u[i] - u[i-1])

1
1
1
1
1


This is the expected result and the execution time was nearly instantaneous.  If we perform the same operation as an array operation, then rather than calculate $u^n_i-u^n_{i-1}\ $ 5 separate times, we can slice the $u$ array and calculate each operation with one command:
<font color='red'> 이것은 예상했던 결과이며 실행 시간이 순간적이였어요. 배열 연산과 같은 연산을 수행하면 $u^n_i-u^n_{i-1}\ $ 을 5번 반복 계산하지 않고 $u$ 배열을 잘라서 명령어 하나로 각 연산을 계산할 수 있습니다.

In [19]:
u[1:] - u[0:-1]

array([1, 1, 1, 1, 1])

What this command says is subtract the 0th, 1st, 2nd, 3rd, 4th and 5th elements of $u$ from the 1st, 2nd, 3rd, 4th, 5th and 6th elements of $u$.
<font color='red'> 이 명령어는 $u$의 1st, 2nd, 3rd, 4th, 5th 및 6th 요소에서 $u$의 0th, 1st, 2nd, 3rd, 4th 및 5th 의 요소를 빼라고 합니다. </font>

### Speed Increases
#### <font color='blue'> 향상된 속도 </font>

For a 6 element array, the benefits of array operations are pretty slim.  There will be no appreciable difference in execution time because there are so few operations taking place.  But if we revisit 2D linear convection, we can see some substantial speed increases.  
<font color='red'>6개 요소를 가진 배열의 경우 배열 연산의 이점은 거의 없다고 봅니다. 작업이 매우 작기에 실행 시간에는 큰 차이가 없습니다. 하지만 2차원 선형 대류를 볼때는 상당히 큰 속도 증가율이 보입니다.

In [9]:
nx = 81
ny = 81
nt = 100
c = 1
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = .2
dt = sigma * dx

x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)

u = numpy.ones((ny, nx)) ##create a 1xn vector of 1's
un = numpy.ones((ny, nx)) 

###Assign initial conditions

u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

With our initial conditions all set up, let's first try running our original nested loop code, making use of the iPython "magic" function `%%timeit`, which will help us evaluate the performance of our code. 
<font color='red'>초기 조건을 설정후 코드의 성능을 평가하는데 도움이되는 iPython "magic"함수 %%timeit 을 사용하여 원래의 중첩 루프 코드를 실행해봅시다. </font>

**Note**: The `%%timeit` magic function will run the code several times and then give an average execution time as a result.  If you have any figures being plotted within a cell where you run `%%timeit`, it will plot those figures repeatedly which can be a bit messy. 
<font color='red'> `%%timeit` 매직 함수는 코드를 여러번 실행후 평균 실행 시간을 결과로 나타냅니다. `%%timeit`을 실행하는 셀(**cell**)안에 숫자가 표기됬다면 그 숫자를 반복적으로 플롯하여 다소 복잡해질수 있습니다.

In [10]:
%%timeit
u = numpy.ones((ny, nx))
u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

for n in range(nt + 1): ##loop across number of time steps
    un = u.copy()
    row, col = u.shape
    for j in range(1, row):
        for i in range(1, col):
            u[j, i] = (un[j, i] - (c * dt / dx * 
                                  (un[j, i] - un[j, i - 1])) - 
                                  (c * dt / dy * 
                                   (un[j, i] - un[j - 1, i])))
            u[0, :] = 1
            u[-1, :] = 1
            u[:, 0] = 1
            u[:, -1] = 1

1 loop, best of 3: 1.83 s per loop


With the "raw" Python code above, the best execution time achieved was 1.94 seconds.  Keep in mind that with these three nested loops, that the statements inside the **j** loop are being evaluated more than 650,000 times.   Let's compare that with the performance of the same code implemented with array operations:
<font color='red'>위의 변형되지 않은 파이썬 코드로 가장 빠른 실행 시간은 1.94 초입니다. 이 세개의 중첩 루프를 사용하면 **j** 루프 내부의 문이 650,000 번 이상 평가됩니다. 배열 작업으로 구현된 동일한 코드의 성능과 비교해 보겠습니다.

In [11]:
%%timeit
u = numpy.ones((ny, nx))
u[int(.5 / dy): int(1 / dy + 1), int(.5 / dx):int(1 / dx + 1)] = 2

for n in range(nt + 1): ##loop across number of time steps 
    un = u.copy()
    u[1:, 1:] = un[1:, 1:] - ((c * dt / dx * (un[1:, 1:] - un[1:, 0:-1])) -
                              (c * dt / dy * (un[1:, 1:] - un[0:-1, 1:])))
    u[0, :] = 1
    u[-1, :] = 1
    u[:, 0] = 1
    u[:, -1] = 1

100 loops, best of 3: 5.38 ms per loop


As you can see, the speed increase is substantial.  The same calculation goes from 1.94 seconds to 5.09 milliseconds.  2 seconds isn't a huge amount of time to wait, but these speed gains will increase exponentially with the size and complexity of the problem being evaluated.  
<font color='red'> 보시다시피, 속도가 눈에띄게 빨라졌습니다. 동일한 계산이 1.94 초에서 5.09 밀리 초로 변화되었습니다. 2초는 긴 시간이 아니지만 평가되는 문제의 크기와 복잡성으로 인해 이러한 속도 향상은 기하 급수적으로 증가합니다.

In [7]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()